In [1]:
import json
import pandas as pd
from sdig.erddap.info import Info
import datetime
import ssl
import dateutil
import constants
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
with open("config/start_config.json", "r+") as site_file:
    site_json = json.load(site_file)
all_start_seconds = 999999999999999999999
all_end_seconds = -1
for site in site_json:

    url = site_json[site]['url']

    info = Info(url)
    
    variables, long_names, units, stardard_names, var_types = info.get_variables()
    not_null = ''
    for var in site_json[site]['variables']:
        not_null = not_null + '&' + var + '!=NaN'
    time_url = url + '.csv?time&orderByMinMax("time")'+not_null
    print(time_url)
    df = pd.read_csv(time_url, skiprows=[1])
    start_date = df['time'].min()
    end_date = df['time'].max()
    starto = dateutil.parser.isoparse(start_date)
    endo = dateutil.parser.isoparse(end_date)
    start_date = datetime.datetime.strftime(starto, constants.d_format)
    end_date = datetime.datetime.strftime(endo, constants.d_format)
    start_date_seconds = starto.timestamp()
    end_date_seconds = endo.timestamp()
    time_marks = Info.get_time_marks(start_date_seconds, end_date_seconds)
    site_json[site]['start_date'] = start_date
    site_json[site]['start_date_seconds'] = start_date_seconds
    if start_date_seconds < all_start_seconds:
        all_start_seconds = start_date_seconds
        all_start = start_date
    if end_date_seconds > all_end_seconds:
        all_end_seconds = end_date_seconds
        all_end = end_date
    site_json[site]['end_date'] = end_date
    site_json[site]['end_date_seconds'] = end_date_seconds
    site_json[site]['long_names'] = long_names
    site_json[site]['units'] = units
    site_json[site]['time_marks'] = time_marks
    site_json[site]['title'] = info.get_title()
    if site_json[site]['depth_name'] == "None":
        site_json[site]['has_depth'] = "false"
    else:
        site_json[site]['has_depth'] = "true"
    if site_json[site]['pressure_name'] == "None":
        site_json[site]['has_pressure'] = "false"
    else:
        site_json[site]['has_pressure'] = "true"
config = {}
config['all_start'] = all_start
config['all_end'] = all_end
config['all_start_seconds'] = all_start_seconds
config['all_end_seconds'] = all_end_seconds
config['sites'] = site_json
print('There are ', len(site_json), ' sites configured.')
print(all_start, all_end)
with open('config/sites.json', 'w') as fp:
    json.dump(config, fp, indent=4)

In [1]:
import tasks
tasks.load_locations()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/workspaceuser/layers/paketo-buildpacks_cpython/cpython/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
  File "/tmp/ipykernel_4099/1598290540.py", line 2, in <module>
    tasks.load_locations()
  File "/home/workspaceuser/app/tasks.py", line 36, in load_locations
    locations_df.to_sql(constants.location_table, constants.postgres_engine, if_exists='replace', index=False, chunksize=1500, method='multi')
  File "/home/workspaceuser/layers/paketo-buildpacks_pip-install/packages/lib/python3.8/site-packages/pandas/core/generic.py", line 2605, in to_sql
    sql.to_sql(
  File "/home/workspaceuser/layers/paketo-buildpacks_pip-install/packages/lib/python3.8/site-packages/pandas/io/sql.py", line 589, in to_sql
    pandas_sql.to_sql(
  File "/home/workspaceuser/layers/paketo-buildpacks_pip-install/packages/lib/python3.8/site-packages/pandas/io/sql.py", line 1393, in to_sql
    table.create()
  File "/home/wor